In [26]:
# numpy provides import array and linear algebra utilities
import numpy as np

# the robotics toolbox provides robotics specific functionality
import roboticstoolbox as rtb

# spatial math provides objects for representing transformations
import spatialmath as sm
import spatialmath.base as smb

# use timeit to benchmark some methods
from timeit import default_timer as timer

# ansitable is a great package for printing tables in a terminal
from ansitable import ANSITable

# 2.1 Utility Functions

In [27]:
# 첫 번째 식은 so(3) -> R^3, 두 번째 식은 R^3 -> so(3), 세 번째 식은 se(3) -> R^6, 네 번째 식은 R^6 -> se(3)로 map 해주는 function. 

![skew symmetric matrix](image/skew_symmetric.png)

In [28]:
# 위 수식을 함수로 나타내기
def vex(mat):
    '''
    Converts a 3x3 skew symmetric matric to a 3 vector
    '''

    return np.array([mat[2, 1], mat[0, 2], mat[1, 0]])

def skew(vec):
    '''
    Converts a 3 vector to a 3x3 skew symmetric matrix
    '''

    return np.array([
        [0, -vec[2], vec[1]],
        [vec[2], 0, -vec[0]],
        [-vec[1], vec[0], 0]
    ])


def vexa(mat):
    '''
    Converts a 4x4 augmented skew symmetric matric to a 6 vector
    '''

    return np.array([mat[0, 3], mat[1, 3], mat[2, 3], mat[2, 1], mat[0, 2], mat[1, 0]])

def skewa(vec):
    '''
    Converts a 6 vector to a 4x4 augmented skew symmetric matrix
    '''

    return np.array([
        [0, -vec[5], vec[4], vec[0]],
        [vec[5], 0, -vec[3], vec[1]],
        [-vec[4], vec[3], 0, vec[2]],
        [0, 0, 0, 0]
    ])
    

In [29]:
# 예시
x = np.array([1,2,3])
print("x: ")
print(x)
print()

print("="*30)
print("skew(x): ")
print()
print(skew(x))
print()

print("="*30)
print("vex(x): ")
print()
print(vex(skew(x)))
print()

y = np.array([1,2,3,4,5,6])
print("="*30)
print("y: ")
print(y)
print()

print("="*30)
print("skewa(y): ")
print()
print(skewa(y))
print()

print("="*30)
print("vexa(y): ")
print()
print(vexa(skewa(y)))
print()

x: 
[1 2 3]

skew(x): 

[[ 0 -3  2]
 [ 3  0 -1]
 [-2  1  0]]

vex(x): 

[1 2 3]

y: 
[1 2 3 4 5 6]

skewa(y): 

[[ 0 -6  5  1]
 [ 6  0 -4  2]
 [-5  4  0  3]
 [ 0  0  0  0]]

vexa(y): 

[1 2 3 4 5 6]



In [30]:
def ρ(tf):
    '''
    The method extracts the rotational component from an SE3
    '''
    return tf[:3, :3]

def τ(tf):
    '''
    The method extracts the translation component from an SE3
    '''
    return tf[:3, 3]

# 2.2 Derivative of a Pure Rotation

![derivativ_rotation](image/derivativ_rotation.png)

In [31]:
import sympy as sp
# 기호 정의
theta = sp.symbols('theta')

def dTRx(theta, flip):
    print("flip: ", flip)
    print("dTRx(theta):")
    print()
    
    Rhx = np.array([
        [0, 0, 0, 0],
        [0, 0, -1, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 0]])
    Trx = rtb.ET.Rx().A(theta)

    if flip:
        print(Rhx.T @ Trx.T)
        return Rhx.T @ Trx.T
    else:
        print(Rhx @ Trx)
        return Rhx @ Trx


def dTRy(theta, flip):
    print("flip: ", flip)
    print("dTRy(theta):")
    print()
    
    Rhy = np.array([
        [0, 0, 1, 0],
        [0, 0, 0, 0],
        [-1, 0, 0, 0],
        [0, 0, 0, 0]])
    Try = smb.troty(theta) #위의 rtb.ET.Ry()랑 같음
    
    if flip:
        print(Rhy.T @ Try.T)
        return Rhy.T @ Try.T
    else:
        print(Rhy @ Try)
        return Rhy @ Try
    
def dTRz(theta, flip):
    print("flip: ", flip)
    print("dTRz(theta):")
    print()
    
    Rhz = np.array([
        [0, -1, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]])
    Trz = smb.trotz(theta)
    
    if flip:
        print(Rhz.T @ Trz.T)
        return Rhz.T @ Trz.T
    else:
        print(Rhz @ Trz)
        return Rhz @ Trz
    
print("="*30)
dTRx(theta,False)
print("="*30)

dTRy(theta,False)
print("="*30)

dTRz(theta,True)
print("="*30)

flip:  False
dTRx(theta):

[[0 0 0 0]
 [0 -sin(theta) -cos(theta) 0]
 [0 cos(theta) -sin(theta) 0]
 [0 0 0 0]]
flip:  False
dTRy(theta):

[[-sin(theta) 0 cos(theta) 0]
 [0 0 0 0]
 [-cos(theta) 0 -sin(theta) 0]
 [0 0 0 0]]
flip:  True
dTRz(theta):

[[-sin(theta) cos(theta) 0 0]
 [-cos(theta) -sin(theta) 0 0]
 [0 0 0 0]
 [0 0 0 0]]


# 2.3 Derivative of a Pure Translation

![Derivative_of_a_Pure_Translation](image/Derivative_of_a_Pure_Translation.png)

In [32]:
def dTtx():
    '''
    Calculates the derivative of an SE3 which is a pure translation along
    the x-axis by amount d
    '''
    # This is the [That_x] matrix in the maths above
    Thx = np.array([
        [0, 0, 0, 1],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ])

    return Thx

def dTty():
    '''
    Calculates the derivative of an SE3 which is a pure translation along
    the y-axis by amount d
    '''
    # This is the [That_y] matrix in the maths above
    Thy = np.array([
        [0, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ])

    return Thy

def dTtz():
    '''
    Calculates the derivative of an SE3 which is a pure translation along
    the z-axis by amount d
    '''
    # This is the [That_z] matrix in the maths above
    Thz = np.array([
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 0]
    ])

    return Thz

# 2.4 The Manipulator Jacobian

In [38]:
# Before going any further, lets remake the panda model we made in the first notebook
E1 = rtb.ET.tz(0.333) 
E2 = rtb.ET.Rz()
E3 = rtb.ET.Ry() 
E4 = rtb.ET.tz(0.316) 
E5 = rtb.ET.Rz()
E6 = rtb.ET.tx(0.0825) 
E7 = rtb.ET.Ry(flip=True) 
E8 = rtb.ET.tx(-0.0825) 
E9 = rtb.ET.tz(0.384) 
E10 = rtb.ET.Rz()
E11 = rtb.ET.Ry(flip=True)
E12 = rtb.ET.tx(0.088) 
E13 = rtb.ET.Rx(np.pi / 2) 
E14 = rtb.ET.tz(0.107)
E15 = rtb.ET.Rz()
panda = E1 * E2 * E3 * E4 * E5 * E6 * E7 * E8 * E9 * E10 * E11 * E12 * E13 * E14 * E15
print(type(panda))

# And make a joint coordinate array q
q = np.array([0, -0.3, 0, -2.2, 0, 2, 0.79])

<class 'roboticstoolbox.robot.ETS.ETS'>


![ETS](image/ETS.png)

In [34]:
def dET(et, η):
    '''
    This method takes an ET and returns the derivative with respect to the joint coordinate
    This is here for convenience
    '''

    if et.axis == 'Rx':
        return dTRx(η, et.isflip)
    elif et.axis == 'Ry':
        return dTRy(η, et.isflip)
    elif et.axis == 'Rz':
        return dTRz(η, et.isflip)
    elif et.axis == 'tx':
        return dTtx()
    elif et.axis == 'ty':
        return dTty()
    else:
        return dTtz()

In [35]:
def dT_j(ets, j, q):
    '''
    This methods calculates the dervative of an SE3 with respect to joint coordinate j
    '''

    # Allocate an identity matrix for the result
    dT = np.eye(4)

    # Find the jth variable et in the ets
    et_j = ets.joints()[j]

    # Loop overs the ETs in the robot ETS
    for et in ets:
        if et == et_j:
            # This ET is variable and corresponds to joint j
            dT = dT @ dET(et, q[et.jindex])
        elif et.isjoint:
            # This ET is a variable joint
            # Use the q array to specify the joint angle for the variable ET
            dT = dT @ et.A(q[et.jindex])
        else:
            # This ET is static
            dT = dT @ et.A()

    return dT

![J_w](image/J_w.png)

In [36]:
def Jω_j(ets, j, q):
    '''
    This method calculates the rotational component of the jth column of the manipulator Jacobian
    '''

    # Calculate the forward kinematics at q
    T = ets.eval(q)

    # Calculate the derivative of T with respect to q_j
    dT = dT_j(ets, j, q)

    Jω = vex(ρ(dT) @ (ρ(T).T))

    return Jω

In [37]:
# Calculate the angular component of the third column of Panda manipulator Jacobian
print(Jω_j(panda, 2, q))

flip:  False
dTRz(theta):

[[ 0. -1.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
[-2.95520207e-01 -6.22199256e-33  9.55336489e-01]
